<font size=8>10k Immunomes Source Code</font>

This code used to create the [10k immunomes website](http://10kimmunomes.ucsf.edu/).

# <font color="gray">Set Up Notebook</font>

## Import Packages

<font color="red">WARNING: the V8 package does not load correctly</font>

In [ ]:
# #require necessary code
require(shiny)
require(shinythemes)
require(digest) 
require(grid)
require(gtable) 
require(MASS) 
require(plyr)
require(reshape2) 
require(scales) 
require(stats) 
require(tibble) 
require(lazyeval)
require(shinyjs) 
require(plotly)

In [ ]:
#require(V8)
#install.packages("V8")

Plotting Packages

In [ ]:
library(ggplot2)
library(plotly)
library(ggthemes)

## download data <small>(if necessary)</small>

Download 10k data

In [ ]:
if( !'newtenkdata.rdata' %in% list.files() ){
    print("Downloading 10k Data")
    download.file(url="https://storage.googleapis.com/focis-tutoria-data/newtenkdata.rdata", destfile="newtenkdata.rdata", method="wget")
    print("Done")
}

Download New Guinea RNA data

In [ ]:
if( !'Guinea_RNA.csv' %in% list.files() ){
    print("Downloading New Guinea RNA Data")
    download.file(url="https://storage.googleapis.com/bakar-data/10k/Guinea_RNA.csv", destfile="Guinea_RNA.csv", method="wget")
    print("Done")
}

Downlaod New Guinea Metabolomics data

In [ ]:
if( !'png_metabolomics.725331.csv' %in% list.files() ){
    print("Downloading New Guinea MassSpec Data")
    download.file(url="https://storage.googleapis.com/bakar-data/10k/png_metabolomics.725331.csv", destfile="png_metabolomics.725331.csv", method="wget")
    print("Done")
}

## Load in Datasets and Resources

 Add the `www` folder to R Shiny's file path. Se that we can display images in the website.

In [ ]:
addResourcePath("www", paste(getwd() , "/www", sep="") )

Load in Guinea Data 

<font color="red">Accidently called a Guinea dataset Gambia, fix this later</font>

In [ ]:
gambia_rna = read.table( "Guinea_RNA.csv",  stringsAsFactors=FALSE, header=TRUE, sep="," )
guinea_mass_spec = read.csv( "png_metabolomics.725331.csv",  stringsAsFactors=FALSE, header=TRUE, sep="," )

Format `guinea_mass_spec` dataset so looks nicer when plotting

In [ ]:
guinea_mass_spec[guinea_mass_spec=="M"] = "Male"
guinea_mass_spec[guinea_mass_spec=="F"] = "Female"
guinea_mass_spec[guinea_mass_spec=="DelayedGroup"] = "Delayed Group"
guinea_mass_spec[guinea_mass_spec=="VaccinatedGroup"] = "Vaccinated Group"

load in a and format `guinea_cibersort` data

In [ ]:
guinea_cibersort = read.csv( "guinea_cibersort.csv",  stringsAsFactors=FALSE, header=TRUE, sep="," )
guinea_cibersort$DAY =  as.numeric( substr( guinea_cibersort$Input.Sample, 6,6 ) )
guinea_cibersort$SUBJECT = substr( guinea_cibersort$Input.Sample, 1,3 ) 

Load in old 10k data

In [ ]:
if( !exists("elisaAnalytes") ){ temp=load('newtenkdata.rdata') }


#  <font color="blue">Initialize Shiny</font> 

This line creates the **<font color="green">user interface</font>**, the "front end" part of the website that everyone sees.

In [ ]:
ui = div() #fluidPage( useShinyjs(), style='padding:5px;'  ) # style='padding:5px;', 

We create the **<font color="purple">server</font>** object then we add code to it one at a time in the followign sections.

In [ ]:
server <- function(input, output, session) { }

These lines add [google analytics](https://analytics.google.com/analytics/web/) features to the website. This makes it so that you can track the users button clicks. 

<font color="orange">NOTE:  I think only 1 of these files is necessary, the other should probably be removed.</font>

In [ ]:
ui = tagAppendChild(ui, tags$head(includeScript("analytics.ucsf.js")) )  
ui = tagAppendChild(ui, tags$head(includeScript("analytics.org.js"))  )

We add the hidden element `page` which controls which page the user is on.

In [ ]:
ui = tagAppendChild(ui,  hidden( selectInput('page', choices = c('home','cytometry','gene', 'lab','rna','antibody','massSpec'), selected = 'home', label=NULL )) )


## <font color="gray">Helper Function: callConcat</font>

<font color="blue">callConcat</font> is a function that combines difference are expressions. [Link to solution](https://codeday.me/en/qa/20190306/8275.html)

In [ ]:
callConcat = function(...) {
    ll <- list(...)
    ll <- lapply(ll, function(X) as.list(X)[-1])
    ll <- do.call("c", ll)
    as.call(c(as.symbol("{"), ll))
}

#  <font color="orange">Titles</font>

This is the information that describes the page at the top of top of the screen. It it all the code that is above the buttons htat switch between the pages.

## <font color="orange">Main Page

In [ ]:
tempHtml = conditionalPanel(title = "homeAbout", condition = "input.page == 'home'",
        div( class="row", style="background-color: rgb(217,234,248,.5)", div( class="col-xs-12 col col-md-10 col-md-offset-1",                                
            img(src='www/Banner_OLDER.jpeg', class="img-responsive")
        )),
        div( class="row", div( class="jumbotron img-responsive",  style="padding: .9em; background-color: rgb(250,250,250)",    #rgb(225, 243, 252)
            #p(" In scientific experiments it’s important to compare experimental results against a control dataset. However, sometimes it’s not possible for a lab to easily obtain data from healthy control subjects. This is especially true in immunology, the study of the immune system, where scientists may run multiple costly tests. We built 10k immunomes so that anyone can get high quality data from healthy subjects using any of the latest scientific methods. In minutes, you will have a graph ready for your paper!"),
            p("In science it’s important to compare experimental results against a control dataset. However, sometimes it’s not possible obtain data from healthy control subjects. This is especially true when studying the immune system, where scientists run multiple costly tests. 10k immunomes allows anyone to obtain high quality data and create graphics using the latest scientific methods."),
            p(tags$small( strong('What is 10k Immunomes?'),
               'The 10,000 Immunomes Project is a reference dataset for human immunology, \r derived from over 10,000 control subjects in the', 
                a(href="http://www.immport.org", "NIAID ImmPort Database",inline = T, target = "_blank"), 
                '. Available data include flow cytometry, CyTOF, multiplex ELISA, gene expression, RNA-Seq, Mass Spectrometry, HAI titers,\r clinical lab tests, HLA type, and others. Click one of the buttons below to view visualizations of all the datasets from that type of study. More information about the website can be found in our', 
                a(href = 'https://www.cell.com/cell-reports/fulltext/S2211-1247(18)31451-7', 'Cell Reports Publication',inline = T, target = "_blank"),'.', 
                a(href = 'mailto:BISC_Helpdesk@niaid.nih.gov', 'Contact us', inline = T, target = "_blank"), 
                'with queries and bug reports. All code is openly available on',
                 a(href = 'https://github.com/pupster90/10k_Immunomes', 'Github', inline = T, target = "_blank"), "and",
                 a(href = 'https://hub.docker.com/r/pupster90/io', 'Docker', inline = T, target = "_blank") ) #'~Last updated 7/25/2019.'
          )
       ))
 )

ui = tagAppendChild(ui, tempHtml )
tempHtml


## <font color="orange">Graph Page

I put all of the pages with graphs inside of one R shiny template. Which then changes depending on what button the user clicks.

First we make the title page template

In [ ]:
tempHtml= conditionalPanel(title = "graph_page_title", condition = "input.page != 'home'",                      
  div( class="row",  div( class="jumbotron", style="padding: .9em; background-size: cover ",
       div( style="display: flex;justify-content: center; position: relative; ",
           h1("Graph title text" , style="text-align: center;", id = "graph_page_header"),
           actionButton("homeBtn", icon = icon("home"), label="", class="btn btn-link", style="  outline: none; padding: 15px; border: 0px; background-color: transparent; font-size: 260%;    ") 
       ), 
       p( tags$small( id="graph_page_text" , "Graph title text" , style="text-align:center;"))
       #h1( "Graph title text" , style="text-align:center; ", id = "graph_page_header"),
       #p(tags$small( id="graph_page_text" , "This is placeholder text"))
  ))
)

ui = tagAppendChild( ui , tempHtml )
#tempHtml


Then we add the code that changes the title page based on which button was clicked.

In [ ]:

temp = quote({
  observeEvent( input$page , {
      
      if( input$page == 'gene' ){
          html(id="graph_page_header", html= "Transcriptomics" ) 
          html(id="graph_page_text", html= "Transcriptomics studies gene expression through the analysis of RNA molecules. This section contains data collected from RNAseq and Microarrays. Until recently Microarrays were the primary method for analyzing gene expression. RNAseq is a newer technique that provides a wider dynamic range and higher sensitivity, albeit at a higher cost. We also provide insilico cell phenotyping data obtained from implementing Cibersort on RNAseq data." ) 
      }
      if( input$page == 'cytometry' ){
          html(id="graph_page_header", html= "Cytometry" ) 
          html(id="graph_page_text", html= "Proteomics is the study of proteins. Proteins are molecular machines that execute the functions of a cell. This section contains flow CyTOF, flow cytometry, and mass spectrometry data. The CyTOF and flow cytometry data provides information on relative abundance of cell phenotypes.  The mass spectrometry data provides scientists with an in depth look at an individual’s metabolome at the time of the sample." )
          
      }
      if( input$page == 'massSpec' ){
          html(id="graph_page_header", html= "Proteomics" ) 
          html(id="graph_page_text", html= "Proteomics is the study of proteins. Proteins are molecular machines that execute the functions of a cell. This section contains CyTOF, flow cytometry, and mass spectrometry data. Flow cytometry is a popular laser-based technology to identify cells. The CyTOF and flow cytometry data contain information on relative abundance of cell phenotypes. Mass spectrometry identifies molecules  through their mass-to-charge ratio, which is measured by analyzing the ‘time-of-flight’ of charged particles. Mass spectrometry data provides scientists with an in depth look at an individual’s metabolome at the time of the sample." ) 
          
      }
      if( input$page == 'lab' ){
          html(id="graph_page_header", html= "Lab Tests" ) 
          html(id="graph_page_text", html= "Lab tests give important information about the current status of a person's metabolism including the health of essential organs, electrolyte and acid/base balance as well as levels of blood glucose and blood proteins. We provide a diverse set of lab tests on blood count, a metabolic panel, and a lipid profile. Blood Count specifies the various amounts of different cell types in the blood. Lipid profiles measure the amount of different types of lipids like cholesterol or triglycerides. Metabolic panels measure the amount to glucose and the electrolyte balance in blood.")
          
      }
      if( input$page == 'rna' ){
          html(id="graph_page_header", html= "RNA-Seq" ) 
          html(id="graph_page_text", html= "Flow cytometry is a popular laser-based technology to analyze cells or particles. It detects and measures physical and chemical characteristics of a population of cells.  In this immunophenotyping technique suspensions of living cells are stained with specific, fluorescently labeled antibodies and then analyzed with a flow cytometer. In the flow cytometer a laser beam is focused on the cell and the light scattered from it is analyzed." ) 
          
      }
      if( input$page == 'antibody' ){
          html(id="graph_page_header", html= "Immunoassays" ) 
          html(id="graph_page_text", html= "Immunoassays are a useful tool for obtaining precise measurements on the concentration of molecules, bacteria, and viruses inside of a sample.  This technique relies on the strong bonding affinity between an antibody and it’s corresponding antigen. Specific antibodies are chosen to ‘grab’ the particle of interest. Immunoassays can also be used to measure the relative effectiveness of an antibody on a specific virus of interest. We provide immunoassays from ELISA, Multiplex ELISA, HAI Titer, and Virus Neutralization studies." ) 
          
      }
  })
})
body(server) = callConcat( body(server), temp )

# <font color="SkyBlue">Buttons</font>

Add <font color="SkyBlue">Buttons</font> to <font color="green">user interface</font>

In [ ]:
tempHtml= div(
    #column(2, actionButton("homeBtn", icon = icon("home"), label="Home", class="btn btn-default btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
    #column(2, actionButton("geneBtn", icon = icon("dna"), label="Sequencing", class="btn btn-success btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
    column(3, actionButton("geneBtn", icon = icon("dna"), label="Transcriptomics", class="btn btn-success btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
    column(3, actionButton("massSpecBtn", icon = icon("cogs"), label="Proteomics", class="btn-primary btn-lg text-center",style="width: 100%"), style="padding: 2px;"),
    column(3, actionButton("antibodyBtn", icon = icon("yahoo"), label="Immunoassays", class="btn btn-warning btn-lg text-center", style="width: 100%"), style="padding: 2px;"),
    #column(2, actionButton("antibodyBtn", icon = icon("yahoo"), label="Immunoassays", class="btn btn-warning btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
    column(3, actionButton("labBtn", icon = icon("vial"), label="Lab Tests", class="btn btn-danger btn-lg text-center", style="width: 100%"),  style="padding: 2px; "),
    p(".", style="color:white;"),
    HTML("<hr>")
)

ui = tagAppendChild(ui, tempHtml )
#tempHtml


In [ ]:
#tempHtml= div(
#    column(2, actionButton("homeBtn", icon = icon("home"), label="Home", class="btn btn-default btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
    #column(2, actionButton("geneBtn", icon = icon("dna"), label="Sequencing", class="btn btn-success btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
#    column(2, actionButton("geneBtn", icon = icon("dna"), label="Genomics", class="btn btn-success btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
#    column(2, actionButton("cytometryBtn", icon = icon("tint"), label="Transciptomics", class="btn-info btn-lg text-center",style="width: 100%"), style="padding: 2px;"),
#    column(2, actionButton("massSpecBtn", icon = icon("atom"), label="Proteomics", class="btn btn-primary btn-lg text-center", style="width: 100%"), style="padding: 2px;"),
#    column(2, actionButton("antibodyBtn", icon = icon("yahoo"), label="Immunoassays", class="btn btn-warning btn-lg text-center", style="width: 100%"), style="padding: 2px; "),
#    column(2, actionButton("labBtn", icon = icon("vial"), label="Lab Tests", class="btn btn-danger btn-lg text-center", style="width: 100%"),  style="padding: 2px; "),
#    p(".", style="color:white;"),
#    HTML("<hr>")
#)

#ui = tagAppendChild(ui, tempHtml )
#tempHtml


Add functions for <font color="skyblue">button clicks</font> to the <font color="purple">server</font>

In [ ]:
temp = quote({
  observeEvent(input$homeBtn, {
    updateSelectInput(session, "page", selected = 'home')
  })
  observeEvent(input$geneBtn, {
    updateSelectInput(session, "page", selected = 'gene')
    updateSelectInput(session, "dataType", selected = 'Microarray: PBMC', choices = c('Microarray: PBMC','Microarray: Whole Blood','RNAseq: T cells','RNAseq: Newborns', 'Cibersort: Newborns'))
    #updateSelectInput(session, "dataType", selected = 'Gene Expression: PBMC', choices = c('Gene Expression: PBMC','Gene Expression: Whole Blood'))   
  })
  observeEvent(input$cytometryBtn, {
    updateSelectInput(session, "page", selected = 'cytometry')
    #updateSelectInput(session, "dataType", selected = 'CyTOF: PBMC', choices = c('CyTOF: PBMC','Flow Cytometry: Whole Blood', 'Flow Cytometry: PBMC'))  
    updateSelectInput(session, "dataType", selected = 'CyTOF: PBMC', choices = c('RNAseq: T cells','RNAseq: Newborns'))
  })
  observeEvent(input$massSpecBtn, {
    updateSelectInput(session, "page", selected = 'massSpec')
    #updateSelectInput(session, "dataType", selected = 'Mass-Spec: Newborns', choices = c('CyTOF: PBMC','Flow Cytometry: Whole Blood', 'Flow Cytometry: PBMC','Mass-Spec: Newborns'))
    updateSelectInput(session, "dataType", selected = 'CyTOF: PBMC', choices = c('CyTOF: PBMC','Flow Cytometry: Whole Blood', 'Flow Cytometry: PBMC','Mass Spectrometry: Newborns'))  
  })
  observeEvent(input$labBtn, {
    updateSelectInput(session, "page", selected = 'lab')
    updateSelectInput(session, "dataType", selected = 'Blood Count', choices = c('Blood Count','Metabolic Panel','Lipid Profile'))
  })
  observeEvent(input$antibodyBtn, {
    updateSelectInput(session, "page", selected = 'antibody')
    updateSelectInput(session, "dataType", selected = 'HAI Titer', choices = c('HAI Titer','ELISA', 'Multiplex ELISA', 'Virus Neutralization Titer'))
  })   
})
body(server) = callConcat( body(server), temp )


# <font color="peru">Body</font>

##  <font color="peru">Body Home</font>

This is the information that is displayed underneath the buttons on the home page.

In [ ]:
tempHtml= conditionalPanel(title = "homeMain", condition = "input.page == 'home'",
   div(class="jumbotron", style="background-color: white; padding: .9em",

        #column(12, column(7,tags$u( h2("Made with ImmPort", class="text-center")  ))),
        column( 7, a(href="http://www.immport.org", img(src='www/Figure_1.png', class="img-responsive"), inline = T, target = "_blank")  ),
        column( 5, 
                p(tags$small("Data from 242 studies and 44,775 subjects was collected from the NIAID Immunology Data and Analysis Portal,",a(href="http://www.immport.org", "ImmPort", inline = T, target = "_blank"),". It includes flow cytometry, CyTOF, mRNA expression, secreted protein levels, clinical lab tests, HAI titers, HLA type, and others. We hand curated the entire contents of ImmPort to filter for normal healthy human control subjects. Each of the data types were systematically processed and harmonized. This data constitutes the largest compendium to date of cellular and molecular immune measurements on healthy normal human subjects."))
              ),
       p(".", style="color:white; padding:0; margin:0; font-size:50%;"), # makes a vertical space,
       HTML("<hr>"),
       
       column( 5,
              tags$u( h2("10,000 Subjects")  ),
              p(tags$small("Below is a table displaying the number of subjects in each dataset. Counts of distinct subjects for whom raw data of each type is represented in the initial release of the 10KIP. Because many subjects contributed multiple measurement types, the totals across all measurement types substantially exceed the number of distinct subjects."))
            ),
       column( 5,
              a(href="http://www.immport.org", img(src='www/Table_1.jpg', class="img-responsive"), inline = T, target = "_blank")
            ),
      # Youtube Video
       div( class="col-xs-12 col-sm-7 col-md-6", style="padding-top: 5px;", 
         div( class="embed-responsive embed-responsive-16by9",
              HTML('<iframe width="200" height="100" src="https://www.youtube.com/embed/iCGEv6MGUVg" class="embed-responsive-item"></iframe>')
             )
          ),
        # Links for external sites
        div( class="col-xs-12 col-sm-12 col-md-6 text-center",
                p(".", style="color:white"), # makes a vertical space
                h3("Learn More"), 
                p(tags$small(
                  a(href = 'http://www.immport.org/immport-open/public/home/home', 'ImmPort Homepage'), br(),
                  a(href = 'https://www.cell.com/cell-reports/fulltext/S2211-1247(18)31451-7', '10,000 Immunomes Paper', target = "_blank"), br(),
                  a(href = 'https://www.cell.com/cell-reports/fulltext/S2211-1247(18)31080-5', 'MetaCyto Cytometry Analysis Paper', target = "_blank"), br(),
                  a(href = 'https://bioconductor.org/packages/release/bioc/html/MetaCyto.html', 'MetaCyto Cytometry Analysis Code', target = "_blank"), br(),
                  a(href = 'https://www.bu.edu/jlab/wp-assets/ComBat/Abstract.html', 'ComBat Batch Correction Algorithm', target = "_blank"), br(),
                  a(href = 'mailto:BISC_Helpdesk@niaid.nih.gov','Contact Us', target = "_blank")
              ))
        )
 



    ),
    div(                        
      column(12,
        tags$hr(),tags$hr(),     
        h3('Cite 10k Immunomes'),
        p("Zalocusky KA, Kan MJ, Hu Z, Dunn P, Thomson E, Wiser J, Bhattacharya S, Butte AJ. The 10,000 Immunomes Project: Building a Resource for Human Immunology. Cell reports. 2019 Oct 9;25(2):513-22. PMID:30304689"  )
        ),
      column(12,
        h3('REFERENCES'),
        p('1) Hu Z, Jujjavarapu C, Hughey JJ, Andorf S, Lee H, Gherardini PF, Spitzer MH, et al. Meta-analysis of Cytometry Data Reveals Racial Differences in Immune Cells. Cell Reports. 2018 Jul 31;24(5):1377-88. ' ),
        p('2) Finak G, Langweiler M, Jaimes M, Malek M, Taghiyar J, Korin Y, et al. Standardizing Flow Cytometry Immunophenotyping Analysis from the Human ImmunoPhenotyping Consortium. Scientific Reports. 2016 Aug 10;6(1):20686.' ),
        p('3) Johnson WE, Li C, Rabinovic A. Adjusting batch effects in microarray expression data using empirical Bayes methods. Biostat. 2007 Jan 1;8(1):118–27. ' ),
        p('4) Irizarry RA, Hobbs B, Collin F, Beazer‐Barclay YD, Antonellis KJ, Scherf U, et al. Exploration, normalization, and summaries of high density oligonucleotide array probe level data. Biostatistics. 2003 Apr 1;4(2):249–64. ' ),
        p('5) Zalocusky KA, Kan MJ, Hu Z, Dunn P, Thomson E, Wiser J, Bhattacharya S, Butte AJ. The 10,000 Immunomes Project: Building a Resource for Human Immunology. Cell reports. 2018 Oct 9;25(2):513-22. PMID:30304689' ),
        p('6) Lee AH, Shannon CP, Amenyogbe N, Bennike TB, Diray-Arce J, Idoko OT, et al. Dynamic molecular changes during the first week of human life follow a robust developmental trajectory. Nature Communications. 2018 March 12;10:1092.' )
        ),
      p(".", style="color:white") # makes a vertical space
   ) #<- end jumbotron
) #<-- end conidtional panel
ui = tagAppendChild(ui, tempHtml )
#tempHtml


## <font color="peru">Body Graph</font>

The code below crete the <font color="peru">Page Body</font>. The is broken up into to basic sections, the **sidebar** and the **main panel**.
* **sidebar:** This is where all the widgets are that control what is graphed
* **main panel:** This small piece of code displays the graph

### <font color="peru">Sidebar

In [ ]:
sidebar = sidebarPanel(  #div( class="col-sm-4", 
        
        #Select which data type to visualize
        h4('Select Data Type:'),
        selectInput('dataType', label = NULL,
                    choices = c('CyTOF: PBMC','ELISA',
                                'Flow Cytometry: Whole Blood', 'Flow Cytometry: PBMC',
                                'Microarray: PBMC','Microarray: Whole Blood',
                                'HAI Titer', 
                                'Blood Count','Metabolic Panel',
                                'Lipid Profile','Multiplex ELISA', 'Virus Neutralization Titer',
                                'RNAseq: T cells', 'RNAseq: Newborns', 'Mass Spectrometry: Newborns', 'Cibersort: Newborns'),
                    selected = 'CyTOF: PBMC'
        ),
        
        ##Conditional panels for analyte selection
        #Which drop-down menu appears will depend on the data type the user selects.
        
        # Analyte Selection Box
    
        conditionalPanel(condition = "input.dataType == 'Mass Spectrometry: Newborns' ", 
                         h4('Enter Analyte'),
                         textInput('mass_spec_analyte', label = NULL,
                                   value = 'cholesterol')
        ),     
        conditionalPanel(condition = "input.dataType == 'Cibersort: Newborns'",
                         h4('Select Analyte:'),
                         selectInput('cibersort_analyte', label = NULL,
                                     choices = colnames( guinea_cibersort )[2:23],
                                     selected = 'Neutrophils')
        ),
        conditionalPanel(condition = "input.dataType == 'CyTOF: PBMC'",
                         h4('Select Analyte:'),
                         selectInput('cytof_pbmc_analyte', label = NULL,
                                     choices = cytof_pbmc_analytes,
                                     selected = 'T_cells')
        ),
        conditionalPanel(condition = "input.dataType == 'ELISA'",
                         h4('Select Analyte:'),
                         selectInput('elisa_analyte', label = NULL,
                                     choices = elisaAnalytes,
                                     selected = 'CXCL10')
        ),
        conditionalPanel(condition = "input.dataType == 'Flow Cytometry: PBMC'",
                         h4('Select Analyte:'),
                         selectInput('flow_pbmc_analyte', label = NULL,
                                     choices =  flow_pbmc_analytes,
                                     selected = 'T_cells')
        ), 
    
        # Normalization Box
        conditionalPanel(condition = "input.dataType == 'CyTOF: PBMC'",
                         checkboxInput('cytof_pbmc_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'ELISA'",
                         checkboxInput('elisa_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'Flow Cytometry: PBMC'",
                         checkboxInput('flow_pbmc_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'Flow Cytometry: Whole Blood'",
                         h4('Select Analyte:'),
                         selectInput('flow_blood_analyte', label = NULL,
                                     choices = flow_blood_analytes,
                                     selected = 'T_cells')
        ),
        conditionalPanel(condition = "input.dataType == 'Microarray: PBMC'",
                         h4('Enter HUGO Gene Symbol:'),
                         textInput('gene_pbmc_analyte', label = NULL,
                                   value = 'CD9')
        ),
        conditionalPanel(condition = "input.dataType == 'Microarray: PBMC'",
                         checkboxInput('gene_pbmc_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'Microarray: Whole Blood'",
                         h4('Enter HUGO Gene Symbol:'),
                         textInput('gene_blood_analyte', label = NULL,
                                   value = 'CD9')
        ),
        conditionalPanel(condition = "input.dataType == 'Microarray: Whole Blood'",
                         checkboxInput('gene_blood_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'HAI Titer'",
                         h4('Select Analyte:'),
                         selectInput('hai_analyte', label = NULL,
                                     choices = haiAnalytes,
                                     selected = haiAnalytes[1])
        ),
        conditionalPanel(condition = "input.dataType == 'HAI Titer'",
                         checkboxInput('hai_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'Multiplex ELISA'",
                         h4('Select Analyte:'),
                         selectInput('mbaaAnalyte', label = NULL,
                                     choices = mbaaAnalytes,
                                     selected = 'CXCL5')
        ),
        conditionalPanel(condition = "input.dataType == 'Multiplex ELISA'",
                         checkboxInput('mbaa_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'Blood Count'",
                         h4('Select Analyte:'),
                         selectInput('cbcAnalyte', label = NULL,
                                     choices = cbcAnalytes,
                                     selected = 'HGB_g_per_dL')
        ),
        conditionalPanel(condition = "input.dataType == 'Blood Count'",
                         checkboxInput('cbc_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'Lipid Profile'",
                         h4('Select Analyte:'),
                         selectInput('flpAnalyte', label = NULL,
                                     choices = flpAnalytes,
                                     selected = flpAnalytes[1])
        ),
        conditionalPanel(condition = "input.dataType == 'Metabolic Panel'",
                         h4('Select Analyte:'),
                         selectInput('cmpAnalyte', label = NULL,
                                     choices = cmpAnalytes,
                                     selected = cmpAnalytes[1])
        ),
        conditionalPanel(condition = "input.dataType == 'Virus Neutralization Titer'",
                         h4('Select Analyte:'),
                         selectInput('vntAnalyte', label = NULL,
                                     choices = vntAnalytes,
                                     selected = 'Measles_Edmonston')
        ),
        conditionalPanel(condition = "input.dataType == 'Virus Neutralization Titer'",
                         checkboxInput('vnt_processing', label = 'Normalized',
                                       value = T)
        ),
        conditionalPanel(condition = "input.dataType == 'RNAseq: T cells' || input.dataType == 'RNAseq: Newborns' ", 
                         h4('Enter HUGO Gene Symbol:'),
                         textInput('T_cell_analyte', label = NULL,
                                   value = 'CD9')
        ),
        conditionalPanel( condition = "input.dataType == 'Mass Spectrometry: Newborns' || input.dataType == 'RNAseq: Newborns' || input.dataType == 'Cibersort: Newborns' ",
             h4('Plot By:'),
             radioButtons( 'newbornPlotBy', label = NULL,
                          choices = c('Days of Life','Sex','Vaccination'),
                          selected = 'Days of Life'
             ) 
        ),
        
        #Select age range of subjects
        conditionalPanel(condition = "input.dataType != 'RNAseq: T cells' && input.dataType != 'RNAseq: Newborns' && input.dataType != 'Mass Spectrometry: Newborns' && input.dataType != 'Cibersort: Newborns'",
                         h4('Age Range:'),
                         sliderInput("ageRange",label = NULL,
                                     min = 0,
                                     max = 100,
                                     value = c(0,100),
                                     dragRange = TRUE,
                                     round = TRUE,
                                     step = 1),
                         
                                                
                         
                         #Select the ethnicities of subjects
                 fluidRow(
                         column( 8, 
                             h4('Ethnicities:'),
                             checkboxGroupInput('race', label = NULL,
                                                choices = c('White', 'Black or African American','Asian','Other'),
                                                selected = c('White', 'Black or African American','Asian','Other')
                             )
                        ),
                         #How to color plot
                         column( 4 ,
                             h4('Plot By:'),
                             radioButtons('colorCode', label = NULL,
                                          choices = c('Age & Sex','Ethnicity','Study'),
                                          selected = 'Age & Sex'
                             )                  
                         )
                     
                     ),
                         
                 fluidRow(   
                                                  #Select male or female subjects
                         column( 12 ,
                             h4('Sex:'),
                             checkboxGroupInput("gender", label = NULL, choices = c('Female','Male'), selected = c('Female','Male'), inline=TRUE  )
                        )
                     )
                         

        )
        
      )
       

#### <font color="gray">Cleaning Code, Failed</font>

```python
sidebar = sidebarPanel(
        
        #Select which data type to visualize
        h4('Select Data Type:'),
        selectInput('dataType', label = NULL,
                    choices = c('CyTOF: PBMC','ELISA',
                                'Flow Cytometry: Whole Blood', 'Flow Cytometry: PBMC',
                                'Gene Expression: PBMC','Gene Expression: Whole Blood',
                                'HAI Titer', 
                                'Lab Tests: Blood Count','Lab Tests: Metabolic Panel',
                                'Lab Tests: Lipid Profile','Multiplex ELISA', 'Virus Neutralization Titer',
                                'RNAseq: T cells', 'Newborn RNAseq'),
                    selected = 'CyTOF: PBMC'
        ),
        
        
        # Select Analyte
        conditionalPanel(condition = "is.element( input.dataType, c('CyTOF: PBMC','ELISA','Flow Cytometry: Whole Blood','Flow Cytometry: PBMC','HAI Titer','Multiplex ELISA','Lab Tests: Blood Count','Lab Tests: Lipid Profile','Lab Tests: Metabolic Panel','Virus Neutralization Titer') )",
                         h4('Select Analyte:'),
                         selectInput('analyte', label = NULL, choices = cytof_pbmc_analytes, selected = 'T_cells')
        ),
        
        # Normalized
        conditionalPanel(condition = "is.element( input.dataType, c('CyTOF: PBMC','ELISA','Flow Cytometry: Whole Blood','Flow Cytometry: PBMC','Gene Expression: Whole Blood','HAI Titer','Multiplex ELISA','Lab Tests: Blood Count','Virus Neutralization Titer','Gene Expression: PBMC') )",
                         checkboxInput('processing', label = 'Normalized', value = T)
        ),    
         
        # Hugo Symbol
        conditionalPanel(condition = "input.dataType == 'RNAseq: T cells'",
                         h4('Enter HUGO Gene Symbol:'),
                         textInput('HUGO', label = NULL, value = 'CD9')
        ),
    
        #Select age range of subjects
        conditionalPanel(condition = "is.element( input.dataType, c('RNAseq: T cells','Newborn RNAseq') )",
                     h4('Age Range:'),
                     sliderInput("ageRange",label = NULL,
                                 min = 0,
                                 max = 100,
                                 value = c(0,100),
                                 dragRange = TRUE,
                                 round = TRUE,
                                 step = 1),
                     #Select male or female subjects
                     h4('Sex:'),
                     checkboxGroupInput("gender", label = NULL,
                                        choices = c('Female','Male'), 
                                        selected = c('Female','Male')),
                     #Select the ethnicities of subjects
                     h4('Ethnicities:'),
                     checkboxGroupInput('race', label = NULL,
                                        choices = c('White', 'Black or African American','Asian','Other'),
                                        selected = c('White', 'Black or African American','Asian','Other')
                     ),
                     #How to color plot
                     h4('Plot By:'),
                     radioButtons('colorCode', label = NULL,
                                  choices = c('Age & Sex','Ethnicity','Study'),
                                  selected = 'Age & Sex'
                     )
        )
      )
```
       

### <font color="gray">Helper</font> <font color="blue">myDownloadButton</font>

I customized R Shiny's download button to make it look nice for the app. The code I used to do this comes [from here](https://stackoverflow.com/questions/49350509/adding-removing-icon-in-downloadbutton-and-fileinput)

In [ ]:
myDownloadButton <- function(outputId, label = "Download", myIcon="download", class = NULL, ... ){
     tags$a(id = outputId, class = paste("btn btn-default shiny-download-link", class),
         href = "", target = "_blank", download = NA, 
         icon(myIcon) , label, ...)
}

### <font color="peru">Graph

<font color="red">Note:</font> in the future uncomment out `numSubjects` to print number of subjects.

In [ ]:
main_panel = mainPanel(   #div( class="col-sm-8",
        plotlyOutput("dataPlot"),
        tags$div(class = 'container2', style = 'width:100; float:right; color:black',textOutput('nSubsText', inline = F)),
        #h6( id='numSubjects', class="pull-right", "Number of Subjects: " ),
        h2( class="col-xs-offset-2 col-md-offset-1", "Download"),
        div(
            myDownloadButton('downloadPlot', label='Image', myIcon="image", class="btn btn-success"),
            myDownloadButton('downloadPlotData', label='Plot Data', myIcon="file-download", class="btn btn-warning"),
            tags$button(  id="downloadAllData", class="btn", style="padding:0"#,  #  id='downloadAllData'
               #HTML("<a  href='https://storage.googleapis.com/bakar-data/10k/png_metabolomics.725331.csv' download ><button class='btn btn-danger'><i class='fa fa-cart-arrow-down'></i> All Data</button></a>")
            )
            #myDownloadButton('downloadAllData', label='All Data', myIcon="cart-arrow-down", class="btn btn-danger")
        )
      )



Putting Graph and SideBar together

In [ ]:
tempHtml  = conditionalPanel( title="graphMain", condition = "input.page != 'home'",             
    sidebarLayout( sidebar, main_panel )             
)    
ui = tagAppendChild(ui, tempHtml )
#tempHtml

### <font color="peru">Data Explanations

Template for Data explanation text

In [ ]:
tempHtml = conditionalPanel( title="info_text", condition = "input.page != 'home'",   
    div( class="jumbotron", style="background-color: white; padding: .9em;",  # padding-left: .9em; padding-top: 0px;",
         p( id="graph_data_info" , '..........' )
   )                        
)
ui = tagAppendChild(ui, tempHtml )


Code that switches what template says based on selected dataset

In [ ]:
temp = quote({
  observeEvent( input$dataType , {
      
      if( input$dataType == 'CyTOF: PBMC' ){
          html(id="graph_data_info", html= "<b>CYTOF Data: </b><small>CyTOF data of healthy human blood samples were downloaded from ImmPort web portal. Every .fcs file was pared down to 5000 events. These .fcs files constitute the “raw” CyTOF:PBMC data. All data were arcsinh transformed. For CyTOF data, the formula f(x) = arcsinh (x/8) was used. Transformation and compensation were done using the preprocessing.batch function in MetaCyto (1). The cell definitions from the Human ImmunoPhenotyping Consortium (2) were used to identify 24 types of immune cells using the searchClster.batch function in MetaCyto. Specifically, each marker in each cytometry panels was bisected into positive and negative regions. Cells fulfilling the cell definitions are identified. For example, the CD14+ CD33+ CD16- (CD16- monocytes) cell subset corresponds to the cells that fall into the CD14+ region, CD33+ region and CD16- region concurrently. The proportion of each cell subsets in the PBMC were then calculated by dividing the number of cells in a subset by the total number of cells in the blood. These steps together produce the “formatted” CyTOF: PBMC data. These data were then batch-corrected with an established empirical Bayes method (3), using study accession for batch and age, sex, and race as known covariates to produce the “formatted and normalized” CyTOF: PBMC data. </small>" ) 
      
      }else if( input$dataType == 'ELISA' ){
          html(id="graph_data_info", html= "<b>ELISA Data: </b><small>Parsed ELISA data were downloaded from ImmPort. Analyte names were standardized to HUGO gene names where appropriate, and measurements were standardized to a common unit of measurement (pg/mL). These steps produced the “formatted” ELISA data. Because ELISA data is low-throughput, and most subjects only have measurements for one analyte, batch correction was conducted with a simple linear model for each analyte, mean correcting by study accession while accounting for age, sex, and race. These steps produced the “formatted and normalized” ELISA data.</small>" ) 
      
      }else if( input$dataType %in% c('Flow Cytometry: Whole Blood','Flow Cytometry: PBMC')  ){
          html(id="graph_data_info", html= "<b>Cytometry Data: </b><small>Meta-analysis of Cytometry data is conducted using the MetaCyto package (1). Briefly, flow cytometry data were downloaded from ImmPort web portal. Every .fcs file was pared down to 5000 events. These .fcs files constitute the “raw” Flow Cytometry:PBMC data. Flow cytometry data from ImmPort were compensated for fluorescence spillovers using the compensation matrix supplied in each fcs file.  All data from ImmPort were arcsinh transformed. For flow cytometry data, the formula f(x) = arcsinh (x/150) was used. Transformation and compensation were done using the preprocessing.batch function in MetaCyto (1). The cell definitions from the Human ImmunoPhenotyping Consortium (2) were used to identify 24 types of immune cells using the searchClster.batch function in MetaCyto. Specifically, each marker in each cytometry panels was bisected into positive and negative regions. Cells fulfilling the cell definitions are identified. For example, the CD14+ CD33+ CD16- (CD16- monocytes) cell subset corresponds to the cells that fall into the CD14+ region, CD33+ region and CD16- region concurrently. The proportion of each cell subsets in the PBMC or whole blood were then calculated by dividing the number of cells in a subset by the total number of cells in the blood. These steps together produce the “formatted” Flow Cytometry: PBMC data. Because the Flow Cytometry data are sparse, batch correction was conducted with a simple linear model for each cell type, mean correcting by study accession while accounting for age, sex, and race. These steps produced the “formatted and normalized” Flow Cytometry data.</small>")
      
      }else if( input$dataType %in% c('Microarray: PBMC','Microarray: Whole Blood') ){
          html(id="graph_data_info", html= "<b>Gene Expression Data: </b><small>Gene expression array data were obtained in three formats. Data in their original formats (.CEL files, series matrix files, etc) constitute the “raw” gene expression data. For data collected on Affymetrix platforms, we utilized the ReadAffy utility in the affy Bioconductor package to read in raw .CEL files. The rma utility was used to conduct Robust Multichip Average (rma) background correction (as in (4)), quantile normalization, and log2 normalization of the data. For data collected on Illumina platforms and stored in the Gene Expression Omnibus (GEO) database, we utilized the getGEO utility in the GEOquery Bioconductor package to read the expression files and the preprocessCore package to conduction rma background correction, quantile normalization, and log2 normalization of the gene expression data. Finally, for data collected on Illumina platforms but not stored in GEO, we utilized the read.ilmn utility of the limma Bioconductor package to read in the data, and the neqc function to rma background correct, quantile normalize, and log2 normalize the gene expression data. In all instances, probe IDs were converted to Entrez Gene IDs. Where multiple probes mapped to the same Entrez Gene ID, the median value across probes was used to represent the expression value of the corresponding gene. The background-corrected and normalized datasets were combined based on common Entrez IDs, missing values were imputed with a k-nearest neighbors algorithm (R package: impute, function: impute.knn) using k = 10 and default values for rowmax, colmax, and maxp. Enter Gene IDs were then converted to HUGO gene names. These steps together produced the “formatted” gene expression files. To create the “formatted and normalized” datasets, we utilized established empirical Bayes algorithm for batch correction (2), compensating for possible batch effects while maintaining potential effects of age, race, and sex across datasets.</small>") 
      
      }else if( input$dataType == 'HAI Titer' ){
          html(id="graph_data_info", html= "<b>HAI Titer Data: </b><small>Parsed HAI data were downloaded from ImmPort. Names were standardized to WHO viral nomenclature where necessary. These steps produced the “formatted” HAI data. Because HAI data is low-throughput, and most subjects only have measurements for one-to-three of the viruses, batch correction was conducted with a simple linear model for each analyte, mean correcting by study accession while accounting for age, sex, and race. These steps produced the “formatted and normalized” HAI data.</small>")
      
      }else if( input$dataType %in%  c('Blood Count','Metabolic Panel','Lipid Profile') ){
          html(id="graph_data_info", html= "<b>Lab Test Data: </b><small>Parsed lab test data were downloaded from ImmPort and organized into three standard panels: Complete Blood Count (CBC), Fasting Lipid Profile (FLP), and Comprehensive Metabolic Panel (CMP). Because FLP and CMP data are derived from only one study, no further standardization was required. These parsed data constitute the 'formatted' lab test data for these two types, and no 'normalized' table is available. CBC data were derived from 12 different studies. As such, names of individual tests as well as units of measurement needed to be standardized for the data to be directly comparable. For example, cells reported as thousands of cells per microliter were variously described as 'K/mi', 'K/', “cells/mm3”, “thou/mcL”, ”per”, “1000/microliter”, “10^3/mm3”, “10^3”, “1e3/uL”, “10*3/ul”, “/uL”, or “10^3 cells/uL”, and the names of assays were comparably variable. These standardization steps produced the “formatted” Lab Test: Blood Count data. These data were then batch corrected with a simple linear model for each analyte, mean correcting by study accession while accounting for age, sex, and race to produce the “formatted and normalized” CBC data.</small>")
      
      }else if( input$dataType == 'Multiplex ELISA' ){
          html(id="graph_data_info", html= "<b>Multiplex ELISA Data: </b><small>Secreted protein data measured on the multiplex ELISA platform were collected from ImmPort studies SDY22, SDY23, SDY111, SDY113, SDY180, SDY202, SDY305, SDY311, SDY312, SDY315, SDY420, SDY472, SDY478, SDY514, SDY515, SDY519, and SDY720. Data were drawn from the ImmPort parsed data tables using RMySQL or loaded into R from user-submitted unparsed data tables. Across the studies that contribute data, there are disparities in terms of the dilution of samples and units of measure in which the data are reported. We corrected for differences in dilution factor and units of measure across experiments and standardized labels associated with each protein as HUGO gene symbols. This step represents the “formatted” Multiplex ELISA data table.  Compensation for batch effects was conducted using an established empirical Bayes algorithm (2), with study accession representing batch and a model matrix that included age, sex, and race of each subject. Data were log2 transformed before normalization to better fit the assumption that the data are normally distributed. The effectiveness of the log2 transform, as well as our batch correction efforts, are detailed in the manuscript associated with this resource (5). This batch-corrected data represents the “formatted and normalized” Multiplex ELISA data. </small>")
      
      }else if( input$dataType == 'Virus Neutralization Titer' ){
          html(id="graph_data_info", html= "<b>Virus Neutralization Data: </b><small>Parsed VNT data were downloaded from ImmPort. Names were standardized to WHO viral nomenclature where necessary. These steps produced the “formatted” VNT data. Because VNT data is low-throughput, and most subjects only have measurements for one-to-three of the viruses, batch correction was conducted with a simple linear model for each analyte, mean correcting by study accession while accounting for age, sex, and race. These steps produced the “formatted and normalized” VNT data.</small>")
      
      }else if( input$dataType %in% c( 'RNAseq: Newborns', 'Mass Spectrometry: Newborns', 'Cibersort: Newborns') ){
          html(id="graph_data_info", html= "<b>Newborn Immunologic Data: </b><small>The newborn RNA-Seq and Mass Spectrometry datasets were downloaded from Immport  (Studies: SDY1256 and SDY1412). The RNAseq data comes from peripheral blood and Mass-Spec data comes from blood plasma. Phenotyping data was obtaiend by from running Cibersort on RNAseq data. The files used from Immport were formatted and normalized by the EPIC Consortium. More information on the study can be found in this <a href=‘https://www.nature.com/articles/s41467-019-08794-x’>Nature Article</a> published by Amy H. Lee, et al. Some minor additional formatting was done for presenting data on 10k Immunomes.</small>") 
      
      }else if( input$dataType == 'RNAseq: T cells' ){
          html(id="graph_data_info", html= "<b>RNA-Seq T Cell Data: </b><small>The T cells' raw sequence read files were download from the Sequence Read Archive (SRA).  The sequence reads are quantified at the gene level using Kallisto. The Transcripts per million (TPM) is used to quantify the transcription levels in T cells.</small>")

      }else{
          html(id="graph_data_info", html= "   ")
      }
  })
})
body(server) = callConcat( body(server), temp )

# <font color="purple">Graph Functions</font>

**<font color="red">NOTE: This code is old, make it better!</font>**

##  <font color="purple">Set data

In [ ]:
temp= quote({
  data <- reactive({
    if(input$dataType == 'Multiplex ELISA' & input$mbaa_processing ==F){return(mbaa_raw[mbaa_raw$age > min(input$ageRange) & mbaa_raw$age < max(input$ageRange)
                                                                                        & mbaa_raw$gender %in% input$gender & mbaa_raw$race %in% input$race,])}
    else if (input$dataType == 'Flow Cytometry: PBMC' & input$flow_pbmc_processing ==F){return(flow_pbmc_raw[flow_pbmc_raw$age > min(input$ageRange) & flow_pbmc_raw$age < max(input$ageRange)
                                                                                                             & flow_pbmc_raw$gender %in% input$gender & flow_pbmc_raw$race %in% input$race,])}
    else if (input$dataType == 'CyTOF: PBMC' & input$cytof_pbmc_processing ==F){return(cytof_pbmc_raw[cytof_pbmc_raw$age > min(input$ageRange) & cytof_pbmc_raw$age < max(input$ageRange)
                                                                                                      & cytof_pbmc_raw$gender %in% input$gender & cytof_pbmc_raw$race %in% input$race,])}
    else if (input$dataType == "Microarray: Whole Blood" & input$gene_blood_processing ==F){return(gene_blood_raw[gene_blood_raw$age > min(input$ageRange) & gene_blood_raw$age < max(input$ageRange)
                                                                                                                       & gene_blood_raw$gender %in% input$gender & gene_blood_raw$race %in% input$race,])}
    else if (input$dataType == "Microarray: PBMC" & input$gene_pbmc_processing ==F){return(gene_pbmc_raw[gene_pbmc_raw$age > min(input$ageRange) & gene_pbmc_raw$age < max(input$ageRange)
                                                                                                              & gene_pbmc_raw$gender %in% input$gender & gene_pbmc_raw$race %in% input$race,])}
    else if (input$dataType == "ELISA" & input$elisa_processing ==F){return(elisa_raw[elisa_raw$age > min(input$ageRange) & elisa_raw$age < max(input$ageRange)
                                                                                      & elisa_raw$gender %in% input$gender & elisa_raw$race %in% input$race,])}
    else if (input$dataType == "HAI Titer"& input$hai_processing ==F){return(hai_raw[hai_raw$age > min(input$ageRange) & hai_raw$age < max(input$ageRange)
                                                                                     & hai_raw$gender %in% input$gender & hai_raw$race %in% input$race,])}
    else if (input$dataType == "Blood Count" & input$cbc_processing ==F){return(cbc_raw[cbc$age > min(input$ageRange) & cbc_raw$age < max(input$ageRange)
                                                                                                   & cbc_raw$gender %in% input$gender & cbc_raw$race %in% input$race,])}
    else if (input$dataType == 'Virus Neutralization Titer' & input$vnt_processing ==F){return(vnt_raw[vnt_raw$age > min(input$ageRange) & vnt_raw$age < max(input$ageRange)
                                                                                                       & vnt_raw$gender %in% input$gender & vnt_raw$race %in% input$race,])}
    else{
      switch(input$dataType,
             "CyTOF: PBMC" = cytof_pbmc[cytof_pbmc$age > min(input$ageRange) & cytof_pbmc$age < max(input$ageRange)
                                        & cytof_pbmc$gender %in% input$gender & cytof_pbmc$race %in% input$race,],
             "ELISA" = elisa[elisa$age > min(input$ageRange) & elisa$age < max(input$ageRange)
                             & elisa$gender %in% input$gender & elisa$race %in% input$race,],
             "Flow Cytometry: PBMC" = flow_pbmc[flow_pbmc$age > min(input$ageRange) & flow_pbmc$age < max(input$ageRange)
                                                & flow_pbmc$gender %in% input$gender & flow_pbmc$race %in% input$race,],
             "Flow Cytometry: Whole Blood" = flow_blood_raw[flow_blood_raw$age > min(input$ageRange) & flow_blood_raw$age < max(input$ageRange)
                                                            & flow_blood_raw$gender %in% input$gender & flow_blood_raw$race %in% input$race,],
             "Microarray: Whole Blood" = gene_blood[gene_blood$age > min(input$ageRange) & gene_blood$age < max(input$ageRange)
                                                         & gene_blood$gender %in% input$gender & gene_blood$race %in% input$race,],
             "Microarray: PBMC" = gene_pbmc[gene_pbmc$age > min(input$ageRange) & gene_pbmc$age < max(input$ageRange)
                                                 & gene_pbmc$gender %in% input$gender & gene_pbmc$race %in% input$race,],
             "HAI Titer" = hai[hai$age > min(input$ageRange) & hai$age < max(input$ageRange)
                               & hai$gender %in% input$gender & hai$race %in% input$race,],
             "HLA Type" = hla[hla$age > min(input$ageRange) & hla$age < max(input$ageRange)
                              & hla$gender %in% input$gender & hla$race %in% input$race,],
             "Multiplex ELISA" = mbaa[mbaa$age > min(input$ageRange) & mbaa$age < max(input$ageRange)
                                      & mbaa$gender %in% input$gender & mbaa$race %in% input$race,],
             "Blood Count" = cbc[cbc$age > min(input$ageRange) & cbc$age < max(input$ageRange)
                                            & cbc$gender %in% input$gender & cbc$race %in% input$race,],
             "Lipid Profile" = flp[flp$age > min(input$ageRange) & flp$age < max(input$ageRange)
                                              & flp$gender %in% input$gender & flp$race %in% input$race,],
             "Metabolic Panel" = cmp[cmp$age > min(input$ageRange) & cmp$age < max(input$ageRange)
                                                & cmp$gender %in% input$gender & cmp$race %in% input$race,],
             'Virus Neutralization Titer' = vnt[vnt$age > min(input$ageRange) & vnt$age < max(input$ageRange)
                                                & vnt$gender %in% input$gender & vnt$race %in% input$race,],
             'RNAseq: T cells' = gene_Tcell_raw,
             'RNAseq: Newborns' = gambia_rna,
             'Mass Spectrometry: Newborns' = guinea_mass_spec, # for debugging, never use this variable anyways
             'Cibersort: Newborns' =  guinea_cibersort # Never actually use this
      )}
  })
})
body(server) = callConcat( body(server), temp )



##  <font color="purple">Set analytes

In [ ]:
temp= quote({
#select the analyte for plotting based on user input
analyte <- reactive({switch(input$dataType,
                              "CyTOF: PBMC" = input$cytof_pbmc_analyte,
                              "ELISA" = input$elisa_analyte,
                              "Flow Cytometry: PBMC" = input$flow_pbmc_analyte,
                              "Flow Cytometry: Whole Blood" = input$flow_blood_analyte,
                              "Microarray: Whole Blood" = input$gene_blood_analyte,
                              "Microarray: PBMC" = input$gene_pbmc_analyte,
                              "Gene Set Enrichment: Whole Blood" = input$gsea_analyte,
                              "HAI Titer" = input$hai_analyte,
                              "HLA Type" = input$hla_analyte,
                              "Multiplex ELISA" = input$mbaaAnalyte,
                              "Blood Count" = input$cbcAnalyte,
                              "Lipid Profile" = input$flpAnalyte,
                              "Metabolic Panel" = input$cmpAnalyte,
                              "Virus Neutralization Titer" = input$vntAnalyte,
                              "RNAseq: T cells" = input$T_cell_analyte,
                              "RNAseq: Newborns" = input$T_cell_analyte, 
                              "Cibersort: Newborns" = input$T_cell_analyte,
                              "Mass Spectrometry: Newborns" = input$mass_spec_analyte
  )})
})
body(server) = callConcat( body(server), temp )


### <font color="gray">Cleaning Code, Failed</font>

```python
#updateSelectInput(session, "dataType", selected = 'RNAseq: T cells', choices = c('RNAseq: T cells','Newborn RNAseq'))
tempHtml=temp= quote({
#select the analyte for plotting based on user input
  analyte <- reactive({
 
      # Get analytes of seleceted dataset and setup input box
      if( input$dataType %in% c('CyTOF: PBMC','ELISA','Flow Cytometry: Whole Blood','Flow Cytometry: PBMC','HAI Titer','Multiplex ELISA','Lab Tests: Blood Count','Lab Tests: Lipid Profile','Lab Tests: Metabolic Panel','Virus Neutralization Titer')  ){
          choices = switch(input$dataType,
                 "CyTOF: PBMC" = cytof_pbmc_analytes,
                 "ELISA" = elisaAnalytes,
                 "Flow Cytometry: PBMC" = flow_pbmc_analytes,
                 "Flow Cytometry: Whole Blood" = flow_blood_analytes,
                 "HAI Titer" = haiAnalytes,
                 "Multiplex ELISA" = mbaaAnalytes,
                 "Lab Tests: Blood Count" = cbcAnalytes,
                 "Lab Tests: Lipid Profile" = flpAnalytes,
                 "Lab Tests: Metabolic Panel" = cmpAnalytes,
                 'Virus Neutralization Titer' = vntAnalytes
                )
          updateSelectInput( session, "analyte", selected = choices[1], choices = choices )
          new_analyte = input$analyte
      }else{ #( input.dataType %in% c('Gene Expression: PBMC','Gene Expression: Whole Blood','RNAseq: T cells','Newborn RNAseq') ){
          new_analyte = input$HUGO
      }  
      
      new_analyte
      
  })
})
boedy(server) = callConcat( body(server), temp ) 
```


##  <font color="purple">Set Up Plot

<font color="orange"> Currently adding Plot types for newborns. Variable being used is `newbornPlotBy`, with types 'Days of Life','Sex','Vaccination'

In [ ]:
temp= quote({
  #Create a plot that can be downloaded. Shiny doesn't allow you to grab output objects (like output$dataPlot) for use in later functions.
  plotInput <- reactive({
    
    #print('made it here 1')
  #if(  input$dataType != "Mass-Spec: Newborns" ){
      
    #Decide based on data type whether the y-axis should be log-scaled
    if (input$dataType == 'Microarray: Whole Blood' || input$dataType == 'Microarray: PBMC' || input$dataType == 'Multiplex ELISA'|| input$dataType == 'RNAseq: T cells'){
      scaleChoice = 'log10'
    } else { 
      scaleChoice = 'identity'
    };
    
    #pull the data
    thisDemo <- reactive({
      if(input$dataType != 'RNAseq: T cells'){
        data()[!is.na(data()[,analyte()]),1:8]
        }else{
        data()[!is.na(data()[,analyte()]),1:3]
      }
      })
    thisData <- reactive({data()[!is.na(data()[,analyte()]), as.character(analyte())]})
    dataBind <- reactive({data.frame(cbind(thisData(),thisDemo()))})
    thisAnalyte <- reactive({analyte()})
    
    #print('made it here 2')  
    #make y axis labels
    ylab <- reactive({ 
      switch( input$dataType ,
             "CyTOF: PBMC" = paste(gsub('_',' ',analyte()),'(percent)'),
             "ELISA"= paste(analyte(), '(Concentration in pg/mL)'),
             "Flow Cytometry: PBMC" = paste(gsub('_',' ',analyte()),'(percent)'),
             "Flow Cytometry: Whole Blood" = paste(gsub('_',' ',analyte()), '(percent)'),
             "Microarray: Whole Blood" = paste(analyte(), '(Expression)'),
             "Microarray: PBMC" = paste(analyte(), '(Expression)'),
             "Gene Set Enrichment: Whole Blood" = paste(analyte()),
             "HAI Titer" = paste(analyte(), '(HAI Titer)'),
             "Multiplex ELISA" = paste(analyte(), '(Concentration in pg/mL)'),
             "Blood Count" = analyte(),
             "Lipid Profile" = analyte(),
             "Metabolic Panel" = analyte(),
             "Virus Neutralization Titer" = paste(analyte(), '(Virus Neutralization Titer)'),
             "RNAseq: T cells" = paste(analyte(), '(Expression)'),
             "RNAseq: Newborns" = paste(analyte(), '(Expression)'),
             "Cibersort: Newborns" = paste(analyte(), '(Expression)'),
             "Mass Spectrometry: Newborns" = paste(analyte(), '(Expression)')
      )
    })
    #generate a title
    title <- reactive({paste(gsub('_',' ',analyte()), 'by Subject')})      
      
  #}  # <-- if(  input$dataType != "Mass-Spec: Newborns"
    #print('made it here 3')
    if( input$dataType != 'RNAseq: T cells' && input$dataType != 'RNAseq: Newborns' && input$dataType != "Mass Spectrometry: Newborns" && input$dataType != 'Cibersort: Newborns' ){ 
      #if plotting by age and sex
      if(input$colorCode == 'Age & Sex'){
        p <- ggplot(data = dataBind(), 
                    aes(x=dataBind()[,'age'], y=dataBind()[,1], colour = factor(dataBind()[,'gender']))) +
          geom_point()  + 
          labs(x = 'Subject Ages', y = ylab(), title = title(), fill = NULL, colour = NULL)
          p +  scale_y_continuous(trans=scaleChoice) + stat_smooth(method=loess,data = dataBind() )+
            theme_gdocs() 
      } 
      
      #if plotting by study
      else if(input$colorCode == 'Study'){
        p <- ggplot(data = dataBind(), aes(x=dataBind()[,'age'], y=dataBind()[,1], colour = factor(dataBind()[,'study_accession']))) +
          geom_point()  + 
          labs(x = 'Subject Ages', y = ylab(), title = title(), fill = NULL, colour = NULL)
        p +  scale_y_continuous(trans=scaleChoice) + stat_smooth(method=loess,data = dataBind(),aes(fill = factor(dataBind()[,'study_accession'])))+
            theme_gdocs() 
      }
      
      #if plotting by ethnicity
      else if(input$colorCode == 'Ethnicity'){
        par(las=1, bty='l', lwd=2, family  = 'sans',cex.axis=1.25, cex.lab=1.25, cex.main=1.75)
        med = median(dataBind()[,1])
        p <- ggplot(data = dataBind(),aes(x = dataBind()[,'race'], y = dataBind()[,1])) 
        p + geom_violin(trim = FALSE, fill = 'deepskyblue4', inherit.aes = T)+
          geom_dotplot(binaxis='y', stackdir='center', dotsize=.2) +
          stat_summary_bin(fun.y = median, geom = 'line', lwd = 1, aes(group = dataBind()[,'race'])) +
          geom_hline(aes(yintercept = med), linetype = 'dashed') +
          labs(x = 'Race', y = ylab(), title = title(), fill = NULL, colour = NULL) + 
          theme_gdocs() 
          #scale_y_continuous(trans=scaleChoice) +   #<-- CAUSES ERRORS!
          # #<-- CAUSES ERRORS!
      }
    }       

    # Plot RNA T-Cell Data
    else if( input$dataType == 'RNAseq: T cells' ){ 
          par(las=1, bty='l', lwd=2, family  = 'sans',cex.axis=1.25, cex.lab=1.25, cex.main=1.75)
          med = median(dataBind()[,1])
          p <- ggplot(data = dataBind(),aes(x = gsub(" CD3.*","",dataBind()[,'CellType']), y = dataBind()[,1])) 
          p + geom_violin(trim = FALSE, fill = 'deepskyblue4', inherit.aes = T)+
            geom_dotplot(binaxis='y', stackdir='center', dotsize=.2) +
            stat_summary_bin(fun.y = median, geom = 'line', lwd = 1, aes(group = dataBind()[,'CellType'])) +
            geom_hline(aes(yintercept = med), linetype = 'dashed') +
            labs(x = 'Cell Type', y = ylab(), title = title(), fill = NULL, colour = NULL) +
            theme(axis.text=element_text(size=16), axis.title=element_text(size=18),
                  plot.title=element_text(size = 20, face = 'bold'), plot.margin=unit(c(.5,.5,.5,.5),'cm'))
            # scale_y_continuous(trans=scaleChoice) #<-- CAUSES ERRORS!!!
    }
    
    # Plot RNA Gambia Data
    else if( input$dataType == 'RNAseq: Newborns' ){
        num = which(gambia_rna$hgnc==analyte() )
        gene_name = gambia_rna[num,2]
        a_row = gambia_rna[num,3:dim(gambia_rna)[2]]
        
        # Put data in data frame
        cols = list()
        to_graph= data.frame( counts = as.numeric(a_row) )
        
        # groups data by age 
        groups = c()
        for( i in names(a_row) ){
          groups= c( groups, substr(i, start=nchar(i)-1, stop=nchar(i) )) 
        }
        to_graph$groups = as.factor( groups )
        
        # create X variable for scatter
        `%+=%` = function(e1,e2) eval.parent(substitute(e1 <- e1 + e2))
        to_graph$X = rnorm( length(a_row) , mean = 0, sd = .15 )
        to_graph$X[ grep("*D0", names(a_row))  ] %+=% 0
        to_graph$X[ grep("*D1", names(a_row))  ] %+=% 1
        to_graph$X[ grep("*D3", names(a_row))  ] %+=% 3
        to_graph$X[ grep("*D7", names(a_row))  ] %+=% 7
        
        # Get data on Vaccination and Sex
        to_graph$sex = guinea_mass_spec$SEX[ match( substr( colnames(a_row), 1,3 ) , guinea_mass_spec$SUBJECT ) ]
        to_graph$treatment = guinea_mass_spec$TREATMENT[ match( substr( colnames(a_row), 1,3 ) , guinea_mass_spec$SUBJECT ) ]

        # Decide how to plot data based on "newbornPlotBy" input value
        main_plot = switch( input$newbornPlotBy ,
             'Days of Life' = ggplot( to_graph, aes(x=X, y=counts, color=groups) ) ,
             'Sex'= ggplot( to_graph, aes(x=X, y=counts, color=sex) ) ,
             'Vaccination' = ggplot( to_graph, aes(x=X, y=counts, color=treatment) )
        )
        
        p = main_plot + geom_point() + theme_gdocs() +
          labs(x='Days Alive', y="Gene Expression (counts)", title=paste(gene_name," Expression in Newborns"), fill = NULL, colour = NULL )      
    }
    
    # Plot Mass Spec Data
    else if( input$dataType == "Mass Spectrometry: Newborns" ){        
        # get column number of analyte
        mass_spec_analyte = input$mass_spec_analyte
        col_num = which( colnames( guinea_mass_spec ) == mass_spec_analyte )[1]
        
        # create mini dataframe of data we want to graph
        to_graph= data.frame( vals=  guinea_mass_spec[,col_num] , groups= as.factor(  guinea_mass_spec$DAY ), sex=guinea_mass_spec$SEX, treatment=guinea_mass_spec$TREATMENT  ) 
        days =  guinea_mass_spec$DAY + rnorm( dim(guinea_mass_spec)[1] , mean = 0, sd = .15 ) 
        to_graph$days = days
        
        # Decide how to plot data based on "newbornPlotBy" input value
        main_plot = switch( input$newbornPlotBy ,
             'Days of Life' = ggplot( to_graph, aes(x=days, y=vals, color=groups) ),
             'Sex'= ggplot( to_graph, aes(x=days, y=vals, color=sex) ),
             'Vaccination' = ggplot( to_graph, aes(x=days, y=vals, color=treatment) ) 
        )
        
        # create and display graph
        p = main_plot + geom_point() + theme_gdocs()  +
        labs(x='Days Alive', y="Molecule Expression", title=paste(mass_spec_analyte," Expression in Newborns"), fill = NULL, colour = NULL )   
    }
        
    # Plot Cibersort Newborns Data
    else if( input$dataType == "Cibersort: Newborns" ){        
        # get cibersort analyte and create graph dataframe
        thisAnalyte = input$cibersort_analyte
        col_num = which( colnames( guinea_cibersort ) == thisAnalyte )[1]
        to_graph= data.frame( vals=  guinea_cibersort[,col_num] , groups= as.factor(guinea_cibersort$DAY) )
        
        # add additional columns to to_graph
        to_graph$days = guinea_cibersort$DAY + rnorm( dim(guinea_cibersort)[1] , mean = 0, sd = .15 ) 
        to_graph$vaccination = guinea_mass_spec[ match( guinea_cibersort$SUBJECT, guinea_mass_spec$SUBJECT), 5 ]
        to_graph$sex = guinea_mass_spec[ match( guinea_cibersort$SUBJECT, guinea_mass_spec$SUBJECT), 6 ]

        # Decide how to plot data based on "newbornPlotBy" input value
        main_plot = switch( input$newbornPlotBy ,
             'Days of Life' = ggplot( to_graph, aes(x=days, y=vals, color=groups) ),
             'Sex'= ggplot( to_graph, aes(x=days, y=vals, color=sex) ),
             'Vaccination' = ggplot( to_graph, aes(x=days, y=vals, color=vaccination) ) 
        )
        
        # create and display graph
        p = main_plot + geom_point() + theme_gdocs()  +
            labs(x='Days Alive', y="Phenotype Expression", title=paste(thisAnalyte," Expression in Newborns"), fill = NULL, colour = NULL )  
    }
        
  }) #<-- plotInput end
})  #<-- Quote end
body(server) = callConcat( body(server), temp )


##  <font color="purple">Number of Subjects</font> <font color="red">Not Done</font>

In [ ]:
temp= quote({
  #Output plot (created below) on the Visualize page
  output$dataPlot <- renderPlotly({ plotInput() })
    
  #Calculate and print the number of distinct subjects in the plot
  #nSubs <- reactive({sum(!is.na(thisData()))})
  #nSubsMessage <- reactive({paste('Distinct Subjects:',nSubs())})
  #output$nSubsText <- renderText({nSubsMessage()})
})
body(server) = callConcat( body(server), temp )

# <font color="green">Download Data</font>

The functions below run when one of the "Download" buttons is clicked.

<font color="red">Note: `Cibersort: Newborns` data is currently filled with dummy variables. Should fix this later

## <font color="green">Download Image</font>

The code that runs when "image" is clicked

In [ ]:
temp= quote({
  #Handle 'Download Plot' button on Visualize page
  output$downloadPlot <- downloadHandler(
    filename = function() {
      paste('10KImmunomes', input$dataType, analyte(), '.pdf', sep='_')
      #paste('10KImmunomes', input$dataType, '.pdf', sep='_')  
    },
    content = function(file) {
      pdf(file, width = 8, height = 5)
      print(plotInput())
      dev.off()
    }
  )
})  #<-- Quote end
body(server) = callConcat( body(server), temp )

## <font color="green">Download Plot Data</font>

The code that runs when "Plot Data" is clicked

In [ ]:
temp= quote({
  output$downloadPlotData <- downloadHandler(
    # Write File Name  
    filename = function() { paste('10KImmunomes', input$dataType, analyte(), '.csv', sep='_') }, # Sys.Date()  <-- can add if u like 
    
    # Create File
    content = function(file) {
      # Get Dataset for file
      dataPlot= switch( input$dataType,
                        'RNAseq: T cells'  =  cbind( data()[,2:3] , data()[, analyte()] ) ,
                        "RNAseq: Newborns"   =  data()[ which(data()$hgnc==analyte()) , ]   ,
                        "Mass Spectrometry: Newborns"=  cbind( data()[,4:8] , data()[, analyte()] ) ,
                        "Cibersort: Newborns"=  cbind( data()[,4:8] , data()[, analyte()] ) ,
                         cbind( data()[,2:5] , data()[, analyte()] ) # <-- All Other Data                       
                      )        
      write.csv( dataPlot, file, row.names = F) # Write File 
    }
  )
})  #<-- Quote end
body(server) = callConcat( body(server), temp )

## <font color="green">Download All Data</font>

The code that runs when "All Data" is clicked

In [ ]:
temp= quote({
    observeEvent( input$dataType , {
        # Get the path to where the data will be downloaded from
        file_path = switch(input$dataType,
              "CyTOF: PBMC" = "https://storage.googleapis.com/bakar-data/10k/CyTOF_PBMC.zip",
              "ELISA" = "https://storage.googleapis.com/bakar-data/10k/ELISA.zip",
              "Flow Cytometry: PBMC" = "https://storage.googleapis.com/bakar-data/10k/Flow_Cytometry_PBMC.zip",
              "Flow Cytometry: Whole Blood" = "https://storage.googleapis.com/bakar-data/10k/Flow_Cytometry_Whole_Blood.zip",
              "Microarray: Whole Blood" = "https://storage.googleapis.com/bakar-data/10k/Gene_Expression_Whole_Blood.zip",
              "Microarray: PBMC" = "https://storage.googleapis.com/bakar-data/10k/Gene_Expression_PBMC.zip",  #"Gene Set Enrichment: Whole Blood" = input$gsea_analyte,
              "HAI Titer" = "https://storage.googleapis.com/bakar-data/10k/HAI_Titer.zip",
              "HLA Type" = "https://storage.googleapis.com/bakar-data/10k/hla.csv",
              "Multiplex ELISA" = "https://storage.googleapis.com/bakar-data/10k/Multiplex_ELISA.zip",
              "Blood Count" = "https://storage.googleapis.com/bakar-data/10k/Lab_Tests_Blood_Count.zip",
              "Lipid Profile" = "https://storage.googleapis.com/bakar-data/10k/lab_test_fasting_lipid_profile.csv",
              "Metabolic Panel" = "https://storage.googleapis.com/bakar-data/10k/lab_test_comprehensive_metabolic_panel.csv",
              "Virus Neutralization Titer" = "https://storage.googleapis.com/bakar-data/10k/Virus_Neutralization_Titer.zip",
              "RNAseq: T cells" = "https://storage.googleapis.com/bakar-data/10k/gene_Tcells_formatted.csv",
              "RNAseq: Newborns" = "https://storage.googleapis.com/bakar-data/10k/Guinea_RNA.csv",
              "Mass Spectrometry: Newborns" = "https://storage.googleapis.com/bakar-data/10k/png_metabolomics.725331.csv",
              "Cibersort: Newborns" = "https://storage.googleapis.com/bakar-data/10k/Guinea_RNA.csv",
        )
        # Change the download button's href to the download path
        html_pre = "<a  href='"
        html_post= "' download ><button class='btn btn-danger'><i class='fa fa-cart-arrow-down'></i> All Data</button></a>" 
        html( id='downloadAllData', html=paste( html_pre, file_path, html_post, sep="" )  )
        
    }) #<-- observeEvent end
})  #<-- Quote end
body(server) = callConcat( body(server), temp )


# <font color="blue">Run  Shiny</font> 

We change the following options so that <font color="blue">RShiny</font> runs at an exposed port and on the computer's public host.

In [ ]:
options(shiny.port = 8888)
options(shiny.host = "0.0.0.0")

We run the app!

In [ ]:
shinyApp(ui = fluidPage( useShinyjs(), style='margin-left:5px; margin-right:5px', ui  ), server = server)
#shinyApp(ui = fluidPage( useShinyjs(), style='padding:5px;', ui  ), server = server)

# <font color="brown">Scratch Paper</font>

## Basic ui and server

ui <- basicPage(
  plotlyOutput("dataPlot")
)

server <- function(input, output) {
  output$dataPlot <-   renderPlotly({     # plot(x, sin(x))  })
    ggplot( to_graph, aes(x=X, y=counts, color=groups) ) + 
      geom_point() +
      labs(x='Days Alive', y="Gene Expression (counts)", title=paste(gene_name," Expression in Newborns"), fill = NULL, colour = NULL ) +
      theme_gdocs()
  })

}


## <font color="purple">Maybe Useful-</font> ShinyJS

The following code can be added to the notebook to add javascript and CSS features to the Shiny application.

``` python
jscode <- "
shinyjs.disableTab = function(name) {
}
"

css <- "
.nav li a.disabled {
background-color: #aaa !important;
color: #333 !important;
cursor: not-allowed !important;
border-color: #aaa !important;
}"
```

After the code is written put a line of code like the one below in the <font color="green">ui</font>

``` python
extendShinyjs(text = jscode)
```

## Maybe Useful: Boostrap Page

``` python
ui = bootstrapPage( h1("howdy Dowdy") )

```